# Grid Search Prototype

In [4]:
import rospy
import time
import numpy as np
import sys
import yaml
import os
from os.path import exists
from ray import tune
import ray
from multiprocessing import Lock
from functools import partial
from config import ProfilerConfig
from pose_trajectory_evaluation import PoseTrajectoryEvaluation
from itertools import combinations, chain

%matplotlib inline
# %matplotlib notebook
%reload_ext autoreload
%autoreload 2

In [43]:
def read_grid_search_params(grid_search_params_file, tune_config):
    if not exists(grid_search_params_file):
        print('Found no grid search parameters.')
        return tune_config

    try:
        with open(grid_search_params_file) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
            for key, value in data.items():
                tune_config[key] = value
    except Exception as e:
        print('[Profiler] Unable to read and set grid search params at {file}'.format(file=grid_search_params_file))
        print(e)
    return tune_config


config_root = '/home/berlukas/Documents/workspace/darpa/src/darpa_subt_mapping/maplab/dependencies/internal/maplab_tools/maplab_profiles/config/'
profiling_grid_search_params_file = 'debug_grid_search'
grid_search_params_file = config_root + profiling_grid_search_params_file + '.yaml'

print(f'Loading from {grid_search_params_file}')
config = {}
config = read_grid_search_params(grid_search_params_file, config)
print(f'Read the following: {config}')

Loading from /home/berlukas/Documents/workspace/darpa/src/darpa_subt_mapping/maplab/dependencies/internal/maplab_tools/maplab_profiles/config/debug_grid_search.yaml
Read the following: {'vi_map_landmark_quality_min_observers': [2, 3, 4], 'vi_map_landmark_quality_max_distance_from_closest_observer': [30, 40]}


In [68]:
def generate_combinations(config):
    values = [np.arange(0,len(x)) for x in config.values()]
    grid = np.meshgrid(*values)
    return np.array(grid).T.reshape(-1,len(values))
        
def print_all_combinations(combinations):
    n_combinations = combinations.shape[0]
    for i in range(0, n_combinations):
        print(f'combination {i+1} is {combinations[i,:]}')
    
def print_combination(config, combination):
    n_params = len(combination)
    key_list = list(config.keys())
    value_list = list(config.values())
    for i in range(0, n_params):
        print(f'param {key_list[i]} has value {value_list[i][combination[i]]}')
    
combinations = generate_combinations(config)
print_combinations(combinations)

print('\n')
comb = combinations[2,:]
print(f'Printing values for {comb}:')
print_combination(config, comb)

combination 1 is [0 0]
combination 2 is [0 1]
combination 3 is [1 0]
combination 4 is [1 1]
combination 5 is [2 0]
combination 6 is [2 1]


Printing values for [1 0]:
param vi_map_landmark_quality_min_observers has value 3
param vi_map_landmark_quality_max_distance_from_closest_observer has value 30


In [87]:
def find_best_config(losses, combinations):
    min_loss = np.amin(losses)
    cur_min_idx =  np.where(losses == min_loss)[0]
    return combinations[cur_min_idx, :][0]

def get_top_n_configs(losses, n, combinations):
    sorted_losses = np.sort(losses)
    best_n = []
    for i in range(0,n):
        cur_min_idx =  np.where(losses == sorted_losses[i])[0]
        best_n.append(combinations[cur_min_idx, :])
    return np.array(best_n).squeeze()
    
losses = np.random.rand((combinations.shape[0]))
best_config = find_best_config(losses, combinations)
print(f'the best config {best_config} is:')
print_combination(config, best_config)

top_3 = get_top_n_configs(losses, 3, combinations)
print(f'top 3 configs: ')
for i in range(0, 3):
    print(f'')
    print_combination(config, top_3[i,:])
    print(top_3[i,:])
    print('--------------------')

the best config [0 1] is:
param vi_map_landmark_quality_min_observers has value 2
param vi_map_landmark_quality_max_distance_from_closest_observer has value 40
top 3 configs: 
[0 1]
--------------------
[1 0]
--------------------
[2 0]
--------------------


In [75]:
np.sort(losses)

array([0.09983753, 0.24708817, 0.33886649, 0.39469686, 0.48295989,
       0.83734724])